# import & get article ID

In [30]:
import requests, json, os, csv, re 
from bs4 import BeautifulSoup as BS

with open ('nhk.json', 'r') as f:
    ids = sorted(x['id'] for x in json.load(f))
    print('oldest ID: %s' % ids[0])
    print('newest ID: %s' % ids[-1])
    print(len(ids))

oldest ID: 185875
newest ID: 198988
546


# Scrape

In [28]:
total_list = []
for i in range(198738, 199200):
    url = f"https://www3.nhk.or.jp/nhkworld/th/news/{i}/"
    response = requests.get(url)
    response.encoding='utf-8'
    if response.status_code == 200:
        dic = {}
        soup = BS(response.text, "html.parser")
        data = soup.find('script',type="application/ld+json")
        data = json.loads(data.text)
        dic['headline'] = data['headline']
        dic['article'] = data['articleBody']
        dic['date'] = data['datePublished']
        dic['url'] = url
        dic['id'] = str(i)
        total_list.append(dic)

with open ('nhk_new.json', 'w') as f:
    with open ('nhk.json', 'r') as g:
        old_list = json.load(g)
        for dic in total_list:
            if dic not in old_list:
                old_list.append(dic)
    json.dump(old_list, f, ensure_ascii=False, indent=4)

# Delete & Rename

In [29]:
if os.path.exists('nhk.json') and os.path.exists('nhk_new.json') and os.path.getsize('nhk_new.json') > os.path.getsize('nhk.json'):
    os.remove('nhk.json')
    os.rename('nhk_new.json', 'nhk.json')

# tokenize

In [5]:
from pythainlp import word_tokenize as wt
def clean(text):
    text = re.sub(r'https?://\S* ', '', text)
    text = re.sub(r'\r?(\n|\t)', ' ', text)
    text = re.sub(r'"+', '', text)
    return wt(text, keep_whitespace=False)

with open('nhk.json') as f:
    data = json.load(f)
with open('nhk_tokenized.txt','w') as f:
    tokenized = [clean(news_dic['article']) for news_dic in data]
    writer = csv.writer(f, lineterminator='\n', delimiter=' ')
    writer.writerows(tokenized)

# Word2Vec


In [15]:
from gensim.models import word2vec
model = word2vec.Word2Vec(tokenized, size=200, min_count=3, window=5, iter=100)

In [16]:
len(model.wv.vocab)

3143

# word frequency

In [6]:
import collections 
from pythainlp import corpus
count = collections.Counter()
stop = collections.Counter()
stops = corpus.thai_stopwords()
for line in tokenized:
    for word in line:
        count[word] += 1
        if word not in stops and word not in ['2', '-']:
            stop[word] += 1

In [7]:
for i in count.most_common(30):
    print(i[0], end=' ')

ที่ ของ ใน และ ได้ ว่า การ ญี่ปุ่น มี จะ นาย เป็น นี้ วัน ให้ เมื่อ สหรัฐ กับ จาก ซึ่ง โดย เพื่อ ไม่ ระบุ คน ตุลาคม ขึ้น ยัง ปี ไป

In [21]:
for i in stop.most_common(30):
    print(i[0], end=' ')

ญี่ปุ่น สหรัฐ ระบุ คน ตุลาคม ปี จีน กล่าวว่า เกาหลีใต้ ที่จะ จังหวัด ประธานาธิบดี เดือน รัฐบาล เจ้าหน้าที่ ชิ พฤศจิกายน ต่าง ๆ เกาหลีเหนือ เรื่อง ประเทศ เมือง ผู้นำ ฮา บริษัท โตเกียว กรุง รัฐมนตรี เรียกร้อง ฮ่องกง

In [27]:
model.wv.most_similar('โอซากา')

[('พารา', 0.5588167905807495),
 ('เหรียญทอง', 0.5159199833869934),
 ('สตรี', 0.4935027062892914),
 ('โง', 0.46441853046417236),
 ('ไซ', 0.4540303945541382),
 ('เชื่อม', 0.45229601860046387),
 ('เจ้าของ', 0.44493910670280457),
 ('022', 0.43452709913253784),
 ('ฮาเนดะ', 0.4340747594833374),
 ('ประทับ', 0.4332934319972992)]

In [ ]:
import numpy as np 
A = np.array([[1,2]])